In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

## For tracing and traking the project over langsmith 
os.environ["LANCHAIN_API_KEY"]=os.getenv("LANCHAIN_API_KEY")
os.environ["LANCHAIN_PROJECT"]=os.getenv("LANCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"]="true"

In [ ]:
from langchain_openai import ChatOpenAI

llm_openai=ChatOpenAI(model="o1-mini")
print(llm_openai)

In [ ]:
result=llm_openai.invoke("How are you?")
print(result.content)

In [6]:
from langchain_groq import ChatGroq
model=ChatGroq(model="qwen-qwq-32b")
model.invoke("Hi My name is Vijay")

AIMessage(content='\n<think>\nOkay, the user introduced themselves as Vijay. I should respond politely. Let me start with a greeting. Maybe say hello back and ask how I can assist him today. Keep it friendly and open-ended so he feels comfortable to ask anything. Let me check for any typos. Alright, that should work.\n</think>\n\nHello Vijay! Nice to meet you. How can I assist you today? Feel free to ask me any questions or let me know if you need help with anything specific! 😊', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 16, 'total_tokens': 119, 'completion_time': 0.241607441, 'prompt_time': 0.004640636, 'queue_time': 1.323755411, 'total_time': 0.246248077}, 'model_name': 'qwen-qwq-32b', 'system_fingerprint': 'fp_28178d7ff6', 'finish_reason': 'stop', 'logprobs': None}, id='run--c50ebe11-23f8-4ef9-9049-95807970a3fc-0', usage_metadata={'input_tokens': 16, 'output_tokens': 103, 'total_tokens': 119})

In [7]:
### Prompt Engineering
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","Answer me as you are lead technical architect of a software company."),
        ("user","{input}")
    ]
)

In [8]:
from langchain_groq import ChatGroq
model=ChatGroq(model="gemma2-9b-it")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022821F9D1D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022821F9DBD0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [16]:
### chaining
chain=prompt|model
chain

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Answer me as you are lead technical architect of a software company.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022821F9D1D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022821F9DBD0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [17]:
res=chain.invoke({"input":"Can you tell me about java?"})
print(res.content)

Java, let me tell you, is a cornerstone of our technology stack and for good reason. It's been a workhorse in the software industry for decades, and its strengths are undeniable. 

Here's the lowdown from a technical architect's perspective:

**The Good:**

* **Robust and Reliable:** Java's strong typing, exception handling, and garbage collection make for incredibly stable and reliable applications. We can build mission-critical systems with confidence knowing Java will handle the heavy lifting.
* **Platform Independence:** "Write once, run anywhere" – that's the magic of the JVM.  Java's bytecode portability allows us to deploy our applications across diverse platforms without rewriting them, saving us time and resources.
* **Large and Active Community:**  Java boasts a massive and vibrant community of developers. This translates to an abundance of readily available libraries, frameworks, and support, making development faster and smoother. Spring, Hibernate, and Struts are just a fe

In [18]:
### Output Parser
from langchain_core.output_parsers import StrOutputParser
op_parser=StrOutputParser()

chain=prompt|model|op_parser
chain

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Answer me as you are lead technical architect of a software company.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022821F9D1D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022821F9DBD0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()

In [19]:
res=chain.invoke({"input":"Can you tell me about java?"})
print(res)

Java, as you know, is a cornerstone of our industry.  It's a robust, object-oriented programming language renowned for its "write once, run anywhere" capability thanks to the Java Virtual Machine (JVM). 

Here's a breakdown from a technical architect's perspective:

**Strengths:**

* **Platform Independence:**  The JVM acts as a bridge, allowing Java code to execute on any platform with a JVM implementation. This portability is invaluable for building scalable, multi-platform applications.
* **Object-Oriented Design:** Java embraces OOP principles like encapsulation, inheritance, and polymorphism, leading to modular, reusable code and better maintainability.
* **Strong Community & Ecosystem:**  Java boasts a massive and active community, meaning ample resources, libraries, frameworks, and support are readily available.

* **Security:** Java implements robust security features like bytecode verification and sandboxing, mitigating security vulnerabilities.
* **Enterprise-Grade:** Java is

In [21]:
### JsonOutputParser 
### we need to change the proompt template
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

op_parser=JsonOutputParser()

prompt=PromptTemplate(
    template="Answer the question {format_instruction} {query}",
    input_variables=["query"],
    partial_variables={"format_instruction":op_parser.get_format_instructions()}
)
prompt

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instruction': 'Return a JSON object.'}, template='Answer the question {format_instruction} {query}')

In [23]:
chain=prompt|model|op_parser
res=chain.invoke({"query":"Can you tell me about java?"})
print(res)

{'name': 'Java', 'type': 'programming language', 'creator': 'James Gosling', 'developer': 'Oracle Corporation', 'year': 1995, 'platform': 'cross-platform', 'object_oriented': True, 'features': ['platform independence', 'garbage collection', 'strong typing', 'multithreading', 'robust error handling'], 'uses': ['enterprise applications', 'web development', 'mobile app development (Android)', 'big data', 'scientific computing'], 'popularity': 'high'}


### With pydantic lets use JsonOutputParser 

In [25]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field

In [ ]:
class Joke(BaseModel):
    que:str=Field(description="question")
    punchline:str=Field(description="anwser")

In [35]:
prompt_py=PromptTemplate(
    template="Answer the question {format_instruction} {query}",
    input_variables=["query"],
    partial_variables={"format_instruction":op_parser.get_format_instructions()}
)
model_py=ChatGroq(model="gemma2-9b-it")
model_py

op_parser_py=JsonOutputParser(pydantic_object=Joke)

chain_py=prompt_py|model_py|op_parser_py



In [36]:
joke_query="Tell a joke for good morning"
res_py=chain_py.invoke({"query":joke_query})
res_py

{'joke': "Why don't scientists trust atoms? Because they make up everything!"}